In [1]:
import time
from functools import partial
from IPython.display import display
from itertools import product

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score

from utils import (
    load, 
    select_datetime_columns,
    transform_datetime_features,
    transform_data,
    make_predictions,
    predict,
    root_mean_squared_error,
    pprint,
    parse_dt
)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/lightgbm/__init__.py:45: FutureWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS will be built by the Apple Clang compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you won't need to install the gcc compiler anymore.
Instead of that, you'll need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", FutureWarning)


In [20]:
for task in range(1, 9):
    df_x, target = load(task, 'train')
    print(task, df_x.shape, select_datetime_columns(df_x))
    print('Memory:', df_x.memory_usage().sum() / (1024 * 1024), 'MB')
    display(df_x.describe())

1 (365, 41) ['datetime_0']
Memory: 0.114250183105 MB


,number_0,number_1,number_2,number_3,number_4,number_5,number_6,number_7,number_8,number_9,...,number_30,number_31,number_32,number_33,number_34,number_35,number_36,number_37,number_38,line_id
count,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,...,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000
mean,0.516228,0.205112,0.608632,0.852131,0.290996,0.666252,0.302688,0.646290,0.645210,0.043836,...,0.084932,0.010959,0.032877,0.032877,0.032877,0.010959,0.010959,0.065753,0.041096,269.131507
std,0.146753,0.127578,0.214655,0.166493,0.171512,0.249354,0.225507,0.256725,0.257029,0.205010,...,0.279162,0.104252,0.178559,0.178559,0.178559,0.104252,0.104252,0.248191,0.198785,159.048922
min,0.000000,0.000000,0.180905,0.000000,0.000000,0.181818,0.000000,0.186120,0.193651,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.420169,0.108137,0.424623,0.802669,0.196531,0.454545,0.121001,0.429022,0.431746,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,131.000000
50%,0.510069,0.178587,0.625628,0.878932,0.394839,0.636364,0.166898,0.630915,0.634921,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,269.000000
75%,0.625153,0.283507,0.791457,0.941849,0.399707,0.909091,0.477051,0.905363,0.911111,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,414.000000
max,0.904415,0.557598,1.000000,1.000000,0.603106,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,535.000000


2 (13958, 10) []
Memory: 1.06498718262 MB


,number_0,number_1,number_2,number_3,number_4,number_5,is_test,line_id
count,1.375900e+04,13958.000000,1.002800e+04,13759.000000,10028.000000,13958.000000,13958.0,13958.000000
mean,4.699422e+05,377.618126,4.028625e+05,23124.075151,32773.118169,6.234919,0.0,9959.730764
std,7.144992e+05,532.443690,6.721022e+05,33451.212991,50953.531324,2.829829,0.0,5740.080848
min,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,1.000000,0.0,0.000000
25%,2.849100e+04,18.943256,1.729400e+04,2531.000000,3903.000000,5.000000,0.0,5007.250000
50%,2.738700e+05,236.020561,1.978200e+05,13792.000000,15492.000000,5.000000,0.0,9950.500000
75%,5.647030e+05,474.502791,4.934702e+05,27273.000000,37483.000000,7.000000,0.0,14894.750000
max,1.050700e+07,7870.182502,8.592943e+06,479683.000000,544460.000000,36.000000,0.0,19933.000000


3 (250000, 42) ['datetime_0']
Memory: 80.1087188721 MB


...

,number_0,number_1,number_2,number_3,number_4,number_5,number_6,number_7,number_8,number_9,...,number_103,number_104,number_105,number_106,number_107,number_108,number_109,number_110,number_111,line_id
count,1.088140e+05,108814.000000,1.088140e+05,108814.000000,108814.000000,108814.000000,58540.000000,58540.000000,108814.000000,108814.000000,...,108814.000000,108814.000000,108814.000000,108814.000000,108814.000000,108814.000000,108814.000000,108814.000000,108814.000000,108814.000000
mean,2.089531e+04,7.277894,3.009173e+04,1.433831,1.439283,1.615486,0.387021,0.507100,0.851647,0.385052,...,0.000249,1.428741,0.000155,2.364034,0.001321,2.184498,0.001492,1.501654,0.001686,90745.314142
std,6.005168e+04,8.518890,8.340291e+04,3.515562,3.517221,2.311491,0.387973,0.424216,2.558449,0.497166,...,0.163858,0.875131,0.166396,1.231600,0.229132,1.294325,0.274199,0.500000,0.251054,52312.033342
min,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.457303,1.000000,-0.422505,1.000000,-0.455627,1.000000,-0.513028,1.000000,-0.250198,0.000000
25%,7.166700e+02,1.666667,1.724872e+03,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,...,-0.200161,1.000000,0.087013,1.000000,-0.126355,1.000000,-0.221501,1.000000,-0.250198,45458.250000
50%,6.333330e+03,5.166667,9.831980e+03,0.083333,0.083333,1.000000,0.246667,0.530000,0.083333,0.000000,...,0.109060,1.000000,0.087013,2.000000,0.008214,2.000000,-0.020566,2.000000,0.251909,90862.000000
75%,2.063333e+04,10.166667,2.876246e+04,1.583333,1.583333,2.166667,0.805000,0.980000,0.666667,1.000000,...,0.109060,1.000000,0.087013,3.000000,0.280334,3.000000,0.272740,2.000000,0.251909,136018.750000
max,5.452659e+06,461.916667,4.702289e+06,441.583333,441.583333,87.166667,1.013333,1.040000,343.250000,4.000000,...,0.109060,4.000000,0.087013,5.000000,0.280334,5.000000,0.272740,2.000000,0.251909,181355.000000


7 (92091, 773) ['datetime_0', 'datetime_1']
Memory: 543.108772278 MB


,number_0,number_1,number_2,number_3,number_4,number_5,number_6,number_7,number_8,number_9,...,number_756,number_757,number_758,number_759,number_760,number_761,number_762,number_763,number_764,line_id
count,92091.000000,92091.000000,92091.000000,92091.000000,92091.000000,92091.000000,92091.000000,92091.000000,92091.000000,8.119200e+04,...,81192.000000,81192.000000,81192.00000,92091.000000,92091.000000,92091.000000,92091.000000,10919.000000,10919.000000,92091.000000
mean,0.828713,43.340229,0.071310,0.097469,45.459068,0.065153,0.071679,0.097979,45.274891,9.303549e+04,...,-0.259719,-0.347573,0.86949,0.470372,0.485672,2.945320,0.998176,0.013434,695.335379,89603.091127
std,0.376762,12.924090,0.257343,0.296597,23.661549,0.246797,0.257957,0.297288,23.478522,1.848573e+05,...,0.875893,0.789310,0.33848,0.725817,0.499797,1.920801,0.042673,0.007679,23.788498,51644.370181
min,0.000000,21.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,-2.000000e+00,...,-1.000000,-1.000000,-1.00000,-1.855410,0.000000,-4.701490,0.000000,0.000573,621.000000,2.000000
25%,1.000000,32.000000,0.000000,0.000000,25.000000,0.000000,0.000000,0.000000,25.000000,1.591800e+04,...,-1.000000,-1.000000,0.88000,0.000000,0.000000,1.711263,1.000000,0.008287,683.000000,44935.500000
50%,1.000000,43.000000,0.000000,0.000000,46.000000,0.000000,0.000000,0.000000,45.000000,4.285350e+04,...,-1.000000,-1.000000,1.00000,0.000000,0.000000,3.127178,1.000000,0.012402,692.000000,89665.000000
75%,1.000000,56.000000,0.000000,0.000000,66.000000,0.000000,0.000000,0.000000,66.000000,1.038852e+05,...,0.800000,0.333333,1.00000,0.933264,1.000000,4.254599,1.000000,0.015941,706.000000,134251.500000
max,1.000000,70.000000,1.000000,1.000000,99.000000,1.000000,1.000000,1.000000,89.000000,8.982638e+06,...,1.000000,1.000000,1.00000,3.484482,1.000000,6.906755,1.000000,0.082402,803.000000,179151.000000


8 (143525, 877) ['datetime_0', 'datetime_1', 'datetime_2', 'datetime_3', 'datetime_4', 'datetime_5', 'datetime_6', 'datetime_7', 'datetime_8', 'datetime_9', 'datetime_10', 'datetime_11', 'datetime_12', 'datetime_13', 'datetime_14', 'datetime_15', 'datetime_16', 'datetime_17', 'datetime_18', 'datetime_19', 'datetime_20', 'datetime_21', 'datetime_22', 'datetime_23', 'datetime_24', 'datetime_25', 'datetime_26', 'datetime_27', 'datetime_28', 'datetime_29', 'datetime_30', 'datetime_31', 'datetime_32', 'datetime_33', 'datetime_34', 'datetime_35', 'datetime_36', 'datetime_37', 'datetime_38', 'datetime_39', 'datetime_40', 'datetime_41', 'datetime_42', 'datetime_43', 'datetime_44', 'datetime_45', 'datetime_46', 'datetime_47', 'datetime_48', 'datetime_49', 'datetime_50', 'datetime_51', 'datetime_52', 'datetime_53', 'datetime_54', 'datetime_55', 'datetime_56', 'datetime_57', 'datetime_58', 'datetime_59', 'datetime_60', 'datetime_61', 'datetime_62', 'datetime_63', 'datetime_64', 'datetime_65', 'da

,number_0,number_1,number_2,number_3,number_4,number_5,number_6,number_7,number_8,number_9,...,number_744,number_745,number_746,number_747,number_748,number_749,number_750,number_751,number_752,line_id
count,143525.000000,143525.000000,143525.000000,143525.000000,143525.000000,112694.000000,143525.000000,143525.000000,143525.000000,143525.000000,...,143525.00000,143525.000000,143525.000000,143525.000000,143525.000000,143525.000000,143525.000000,143525.000000,143525.000000,143525.000000
mean,102777.716342,0.268741,0.999993,61369.942519,0.017844,2.119585,10.599652,0.034226,0.008314,10.599324,...,1.98637,1.936601,1.909512,2.150066,2.379437,2.492319,2.059207,2.151063,2.204459,102509.228065
std,59157.641372,0.443306,0.002640,93924.423484,0.132383,0.399429,5.759998,0.014008,0.003668,5.759342,...,6.00990,5.942138,5.901065,6.215626,6.511510,6.650081,6.052867,6.090627,6.120477,59143.878833
min,1.000000,0.000000,0.000000,10000.000000,0.000000,1.100000,1.000000,0.006731,0.001362,1.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51612.000000,0.000000,1.000000,13000.000000,0.000000,2.100000,6.000000,0.024042,0.005634,6.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51288.000000
50%,102887.000000,0.000000,1.000000,24000.000000,0.000000,2.110000,11.000000,0.032673,0.007873,11.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,102546.000000
75%,154013.000000,1.000000,1.000000,67000.000000,0.000000,2.150000,16.000000,0.043373,0.010692,16.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,153724.000000
max,205037.000000,1.000000,1.000000,600000.000000,1.000000,3.400000,20.000000,0.195793,0.049748,20.000000,...,35.50000,35.500000,35.500000,35.500000,35.500000,35.500000,35.500000,35.500000,35.500000,205036.000000


if < 500 MB

In [109]:
df_x, target = load(4, 'train')

In [110]:
datetime_columns = select_datetime_columns(df_x)
old_features = list(df_x.columns)
df_x = transform_datetime_features(df_x, datetime_columns)
df_dt_features = [col for col in df_x.columns if col not in old_features]
df_dt_features

['dt_number_weekday_datetime_0',
 'dt_number_month_datetime_0',
 'dt_number_day_datetime_0',
 'dt_number_hour_datetime_0',
 'dt_number_hour_of_week_datetime_0',
 'dt_number_minute_of_day_datetime_0',
 'dt_number_weekday_datetime_1',
 'dt_number_month_datetime_1',
 'dt_number_day_datetime_1',
 'dt_number_hour_datetime_1',
 'dt_number_hour_of_week_datetime_1',
 'dt_number_minute_of_day_datetime_1',
 'dt_number_weekday_datetime_2',
 'dt_number_month_datetime_2',
 'dt_number_day_datetime_2',
 'dt_number_hour_datetime_2',
 'dt_number_hour_of_week_datetime_2',
 'dt_number_minute_of_day_datetime_2']

In [111]:
dt_column = datetime_columns[0]
dt_column

'datetime_0'

In [112]:
def shift_by_datetime(df_x, dt_column_name):
    df_shifted = df_x.sort_values(dt_column_name).shift(-1)
    df_shifted.columns = [f'{col}__{dt_column_name}__1' for col in df_shifted.columns]
    return df_shifted

In [113]:
def create_lag_features(df_x, datetime_columns):
    tables = [df_x, ]
    for dt_column in datetime_columns:
        df_shifted = shift_by_datetime(df_x, dt_column)
        tables.append(df_shifted)
    return pd.concat(tables, axis=1)

In [114]:
# def hstack(df_left, df_right, result_columns=None):
#     if result_columns is None:
#         df_stacked = pd.concat([df_left, df_right], axis=1)
#     else:
#         df_stacked = pd.concat([df_left, df_right], axis=1)
#         for col in result_columns:
#             if col not in df_stacked.columns:
#                 df_stacked[col] = np.nan
#     return df_stacked

In [115]:
create_lag_features(df_x, datetime_columns[:1]).head()

,number_0,number_1,number_2,number_3,number_4,number_5,number_6,number_7,number_8,number_9,...,dt_number_day_datetime_1__datetime_0__1,dt_number_hour_datetime_1__datetime_0__1,dt_number_hour_of_week_datetime_1__datetime_0__1,dt_number_minute_of_day_datetime_1__datetime_0__1,dt_number_weekday_datetime_2__datetime_0__1,dt_number_month_datetime_2__datetime_0__1,dt_number_day_datetime_2__datetime_0__1,dt_number_hour_datetime_2__datetime_0__1,dt_number_hour_of_week_datetime_2__datetime_0__1,dt_number_minute_of_day_datetime_2__datetime_0__1
0,26.0,NaN,211.67,16.000000,0.384615,123.0,1.000000,1.0,26.0,NaN,...,10.0,0.0,96.0,0.0,3.0,4.0,5.0,0.0,72.0,0.0
1,1.0,14.0,784.98,14.000000,0.000000,0.0,0.000000,0.0,1.0,2395.47,...,13.0,0.0,0.0,0.0,3.0,4.0,5.0,0.0,72.0,0.0
2,24.0,0.0,903.37,22.750000,0.291667,148.0,0.666667,1.0,1.0,0.00,...,23.0,0.0,72.0,0.0,3.0,4.0,5.0,0.0,72.0,0.0
3,26.0,29.0,2744.07,6.333333,0.827586,3323.0,0.500000,3.0,NaN,10162.39,...,14.0,0.0,0.0,0.0,4.0,10.0,13.0,0.0,96.0,0.0
4,15.0,NaN,1579.78,18.666667,0.066667,99.0,0.000000,0.0,14.0,NaN,...,17.0,0.0,96.0,0.0,3.0,4.0,5.0,0.0,72.0,0.0


In [116]:
dt_feature = df_dt_features[0]
dt_feature

'dt_number_weekday_datetime_0'

In [117]:
def aggregate(df_x, by, aggregations=('sum', 'count', 'std', 'min', 'max')):
    df_aggregated = df_x.groupby(by).agg(aggregations)
    df_aggregated.columns = [f'{col[0]}_aggby_{by}_{col[1]}' for col in df_aggregated.columns.values]
    return df_aggregated

In [118]:
def merge_with_aggregated(df_x, df_aggregated, on):
    return df_x.merge(df_aggregated, left_on=on, right_index=True, how='left')

In [119]:
def create_aggregations(df_x, by):
    for feature in by:
        df_aggregated = aggregate(df_x, feature)
        df_x = merge_with_aggregated(df_x, df_aggregated, feature)
    return df_x

In [120]:
create_aggregations(df_x, by=['dt_number_weekday_datetime_0'])

,number_0,number_1,number_2,number_3,number_4,number_5,number_6,number_7,number_8,number_9,...,dt_number_hour_of_week_datetime_2_aggby_dt_number_weekday_datetime_0_sum,dt_number_hour_of_week_datetime_2_aggby_dt_number_weekday_datetime_0_count,dt_number_hour_of_week_datetime_2_aggby_dt_number_weekday_datetime_0_std,dt_number_hour_of_week_datetime_2_aggby_dt_number_weekday_datetime_0_min,dt_number_hour_of_week_datetime_2_aggby_dt_number_weekday_datetime_0_max,dt_number_minute_of_day_datetime_2_aggby_dt_number_weekday_datetime_0_sum,dt_number_minute_of_day_datetime_2_aggby_dt_number_weekday_datetime_0_count,dt_number_minute_of_day_datetime_2_aggby_dt_number_weekday_datetime_0_std,dt_number_minute_of_day_datetime_2_aggby_dt_number_weekday_datetime_0_min,dt_number_minute_of_day_datetime_2_aggby_dt_number_weekday_datetime_0_max
0,26.0,NaN,211.67,16.000000,0.384615,123.0,1.000000,1.0,26.0,NaN,...,1364064,20395,46.212564,0,144,0,20395,0.0,0,0
1,1.0,14.0,784.98,14.000000,0.000000,0.0,0.000000,0.0,1.0,2395.47,...,1146984,17086,42.809628,0,144,0,17086,0.0,0,0
2,24.0,0.0,903.37,22.750000,0.291667,148.0,0.666667,1.0,1.0,0.00,...,1219392,19374,45.727720,0,144,0,19374,0.0,0,0
3,26.0,29.0,2744.07,6.333333,0.827586,3323.0,0.500000,3.0,NaN,10162.39,...,1219392,19374,45.727720,0,144,0,19374,0.0,0,0
4,15.0,NaN,1579.78,18.666667,0.066667,99.0,0.000000,0.0,14.0,NaN,...,807672,14217,42.434107,0,144,0,14217,0.0,0,0
5,27.0,0.0,2474.22,6.000000,0.222222,980.0,0.000000,3.0,2.0,0.00,...,841704,12979,40.743411,0,144,0,12979,0.0,0,0
6,19.0,NaN,1101.63,27.800000,0.105263,160.0,1.000000,0.0,19.0,NaN,...,1364064,20395,46.212564,0,144,0,20395,0.0,0,0
7,11.0,NaN,966.62,16.500000,0.181818,256.0,0.000000,2.0,10.0,NaN,...,1146984,17086,42.809628,0,144,0,17086,0.0,0,0
8,18.0,NaN,0.00,18.000000,0.111111,146.0,0.000000,3.0,2.0,NaN,...,1364064,20395,46.212564,0,144,0,20395,0.0,0,0
9,11.0,NaN,2198.98,9.285714,0.454545,61.0,0.000000,2.0,11.0,NaN,...,1024824,15896,47.143197,0,144,0,15896,0.0,0,0
